# CAH 30503 — Week 5: Deploy

**Theme**: From "it works for me" to "it works for anyone with a URL."

---

Two upgrades today. First: replace `gr.Interface` with `gr.Blocks` — better layout, error handling, and tabs. Second: extract your code into three files and deploy to Hugging Face Spaces.

By the end of class, you'll have a **public URL**.

## Setup

In [ ]:
!pip install -q transformers torch gradio

import torch
import gradio as gr
from transformers import pipeline

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
print(f"Gradio version: {gr.__version__}")
print("Setup complete!")

In [ ]:
# Load pipelines (same as Week 4 — modify for YOUR project)
summarizer = pipeline("summarization", device=device)
ner = pipeline("ner", aggregation_strategy="simple", device=device)
sentiment = pipeline("sentiment-analysis", device=device)

print("Pipelines loaded.")

---

## Activity 1: Upgrade to gr.Blocks

### The Structural Shift

`gr.Interface` was the training wheels. `gr.Blocks` gives you full control.

| `gr.Interface` | `gr.Blocks` |
|----------------|-------------|
| Components declared inside one call | Components created separately |
| Layout is automatic | Layout uses `gr.Row()`, `gr.Column()` |
| Submit button is automatic | You create `gr.Button()` and wire it |
| One function per app | As many functions as you want |

In [ ]:
# DEMO: The same sentiment analyzer — now with gr.Blocks

def analyze_sentiment(text):
    if not text or not text.strip():
        return "Please enter some text to analyze."
    result = sentiment(text)
    return f"{result[0]['label']} (confidence: {round(result[0]['score'], 3)})"

with gr.Blocks() as demo_blocks:
    # Header
    gr.Markdown("# Sentiment Analyzer")
    gr.Markdown("Analyze the emotional tone of any text.")

    # Layout: input and output side by side
    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(label="Your Text", placeholder="Type or paste text here...", lines=5)
            analyze_btn = gr.Button("Analyze", variant="primary")
        with gr.Column():
            output_text = gr.Textbox(label="Sentiment")

    # Wire the button to the function
    analyze_btn.click(fn=analyze_sentiment, inputs=input_text, outputs=output_text)

demo_blocks.launch(share=True)

### The Pattern

```python
with gr.Blocks() as demo:
    # 1. Create components
    gr.Markdown("# Title")
    input_box = gr.Textbox()
    output_box = gr.Textbox()
    btn = gr.Button("Go")

    # 2. Wire events
    btn.click(fn=my_function, inputs=input_box, outputs=output_box)
```

**Layout tools:**
- `gr.Row()` — puts things side by side
- `gr.Column()` — stacks things vertically
- `gr.Tab("name")` — organizes tools into tabs
- `gr.Accordion("name")` — collapsible section

### Error Handling

Your Week 4 app probably crashed or gave garbage when someone:
- Submitted with no input
- Pasted an extremely long document
- Entered something unexpected

**The fix: guard clauses at the top of your function.**

```python
def my_function(text):
    # Guard 1: Empty input
    if not text or not text.strip():
        return "Please enter some text to analyze."

    # Guard 2: Too long
    if len(text) > 5000:
        return f"Text is too long ({len(text)} characters). Please keep it under 5,000."

    # Guard 3: Catch unexpected errors
    try:
        # ... your pipeline code ...
        return result
    except Exception as e:
        return "Something went wrong. Try a different text."
```

**Design question**: What should the error message say? Remember — the person reading it doesn't know what a pipeline is.

In [ ]:
# DEMO: A compound function with error handling + multi-tool tabbed interface

def analyze_article_safe(text):
    """Analyze an article with input validation."""
    if not text or not text.strip():
        return "Please enter some text to analyze."
    if len(text.split()) < 20:
        return "Text is too short for analysis. Please enter at least a few sentences."
    if len(text) > 5000:
        return f"Text is too long ({len(text)} characters). Please keep it under 5,000."

    try:
        summary_out = summarizer(text, max_length=80, min_length=20)
        summary = summary_out[0]["summary_text"]

        entities_out = ner(text)
        seen = set()
        unique = []
        for ent in entities_out:
            if ent["word"] not in seen:
                seen.add(ent["word"])
                unique.append(f"{ent['word']} [{ent['entity_group']}]")

        sent_out = sentiment(summary)
        sent_label = sent_out[0]["label"]
        sent_score = round(sent_out[0]["score"], 3)

        result = f"SUMMARY:\n{summary}\n\n"
        result += f"KEY ENTITIES:\n{', '.join(unique[:10]) if unique else 'None found'}\n\n"
        result += f"OVERALL SENTIMENT: {sent_label} (confidence: {sent_score})"
        return result
    except Exception as e:
        return "Something went wrong during analysis. Try a different text."


def quick_sentiment(text):
    if not text or not text.strip():
        return "Please enter some text."
    try:
        result = sentiment(text)
        return f"{result[0]['label']} (confidence: {round(result[0]['score'], 3)})"
    except Exception as e:
        return f"Error: {e}"


# Multi-tool tabbed interface
with gr.Blocks() as app:
    gr.Markdown("# AI Text Toolkit")
    gr.Markdown("Analyze articles or check sentiment.")

    with gr.Tab("Article Analyzer"):
        gr.Markdown("Paste an article to get a summary, key entities, and sentiment.")
        with gr.Row():
            with gr.Column():
                art_input = gr.Textbox(label="Article Text", placeholder="Paste article...", lines=8)
                art_btn = gr.Button("Analyze Article", variant="primary")
            with gr.Column():
                art_output = gr.Textbox(label="Analysis", lines=10)
        art_btn.click(fn=analyze_article_safe, inputs=art_input, outputs=art_output)

    with gr.Tab("Quick Sentiment"):
        gr.Markdown("Check the sentiment of any text — fast.")
        sent_input = gr.Textbox(label="Text", placeholder="Type anything...", lines=3)
        sent_btn = gr.Button("Check Sentiment", variant="primary")
        sent_output = gr.Textbox(label="Result")
        sent_btn.click(fn=quick_sentiment, inputs=sent_input, outputs=sent_output)

app.launch(share=True)

### Plan Your Upgraded Interface

Before building, sketch your design:

- **Title and description**: 

- **Layout** *(side by side? stacked? tabs?)*:

- **Error cases I need to handle**:
  1. 
  2. 
  3. 

- **What I'm improving from Week 4**: *(be specific)*



In [ ]:
# BUILD YOUR gr.Blocks APP
# Modify the template below for YOUR project

# with gr.Blocks() as my_app:
#     gr.Markdown("# Your App Title")
#     gr.Markdown("One sentence describing what this does.")
#
#     with gr.Row():
#         with gr.Column():
#             my_input = gr.Textbox(label="Input", placeholder="...", lines=5)
#             my_btn = gr.Button("Go", variant="primary")
#         with gr.Column():
#             my_output = gr.Textbox(label="Output", lines=8)
#
#     my_btn.click(fn=your_function, inputs=my_input, outputs=my_output)
#
# my_app.launch(share=True)

print("Uncomment and modify the code above for YOUR app.")

---

## Activity 2: The Three-File Deployment Pattern

Your app runs in this notebook. Close the notebook and it's gone. To deploy, we extract it into **three files** that Hugging Face Spaces can run.

### File 1: `app.py`
Your application code — same code from above, extracted into a standalone Python file.

**Four things change from notebook to app.py:**
1. All imports at the top
2. Inline device detection (no external imports)
3. Pipelines load at module level (top of file, not inside functions)
4. `if __name__ == "__main__": demo.launch()` at the bottom

### File 2: `requirements.txt`
Every package your app needs. The Space starts with nothing installed.

### File 3: `README.md`
YAML front matter that tells Hugging Face how to run your app.

In [ ]:
# EXAMPLE: What a complete app.py looks like

example_app_py = '''"""Sentiment Analyzer — deployed to Hugging Face Spaces."""

import gradio as gr
from transformers import pipeline

classifier = pipeline("sentiment-analysis")


def analyze(text: str) -> str:
    if not text or not text.strip():
        return "Please enter some text."
    result = classifier(text)
    return f"{result[0]['label']} (confidence: {round(result[0]['score'], 3)})"


demo = gr.Interface(
    fn=analyze,
    inputs=gr.Textbox(label="Text", placeholder="Enter text...", lines=3),
    outputs=gr.Textbox(label="Sentiment"),
    title="Sentiment Analyzer",
    description="Analyze the sentiment of your text.",
    examples=[
        ["I love this product!"],
        ["This is terrible."],
        ["The weather is okay today."],
    ],
)

if __name__ == "__main__":
    demo.launch()
'''

print("=== Example app.py ===")
print(example_app_py)

In [ ]:
# EXAMPLE: requirements.txt

example_requirements = '''gradio
transformers
torch
'''

print("=== Example requirements.txt ===")
print(example_requirements)
print("Rule: for every 'import X' in app.py, X must be in requirements.txt")

In [ ]:
# EXAMPLE: README.md with YAML front matter

example_readme = '''---
title: Sentiment Analyzer
emoji: 🎯
colorFrom: blue
colorTo: green
sdk: gradio
sdk_version: "5.0.0"
app_file: app.py
pinned: false
---

# Sentiment Analyzer

Analyze the sentiment of any text.
'''

print("=== Example README.md ===")
print(example_readme)
print("Critical YAML fields: sdk: gradio, app_file: app.py")
print("Everything else is metadata.")

### Build YOUR Deployment Files

Modify the cells below to match YOUR app. You can also direct Claude Code: *"Extract this notebook code into three deployment files for Hugging Face Spaces."*

In [ ]:
# YOUR app.py
# Modify this to contain YOUR app code

my_app_py = '''"""YOUR APP TITLE — deployed to Hugging Face Spaces."""

import gradio as gr
import torch
from transformers import pipeline

# Inline device detection
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load pipelines at module level
# CHANGE THESE to match your project:
# my_pipeline = pipeline("task-name", device=device)


def my_function(text):
    """YOUR FUNCTION — paste from above and modify."""
    if not text or not text.strip():
        return "Please enter some text."
    # YOUR CODE HERE
    return "Replace this with your actual function"


# YOUR INTERFACE — gr.Blocks or gr.Interface
demo = gr.Interface(
    fn=my_function,
    inputs=gr.Textbox(label="Input", lines=5),
    outputs=gr.Textbox(label="Output"),
    title="Your App Title",
    description="What this does.",
)

if __name__ == "__main__":
    demo.launch()
'''

print("=== Your app.py ===")
print(my_app_py)

In [ ]:
# YOUR requirements.txt

my_requirements = '''gradio
transformers
torch
'''

print("=== Your requirements.txt ===")
print(my_requirements)

In [ ]:
# YOUR README.md
# Change title, emoji, and description

my_readme = '''---
title: Your App Title
emoji: 📝
colorFrom: blue
colorTo: green
sdk: gradio
sdk_version: "5.0.0"
app_file: app.py
pinned: false
---

# Your App Title

Description of what your app does.
'''

print("=== Your README.md ===")
print(my_readme)

In [ ]:
# Save your three files to Google Drive (or download them)
import os

deploy_dir = "my-space"
os.makedirs(deploy_dir, exist_ok=True)

for filename, content in [("app.py", my_app_py),
                          ("requirements.txt", my_requirements),
                          ("README.md", my_readme)]:
    filepath = os.path.join(deploy_dir, filename)
    with open(filepath, "w") as f:
        f.write(content)
    print(f"Saved: {filepath}")

print(f"\nAll three files saved to: {deploy_dir}/")
print("Download these and upload them to your Hugging Face Space.")

---

## Activity 3: Deploy to Hugging Face Spaces

### Step by Step

1. Go to [huggingface.co/new-space](https://huggingface.co/new-space)
2. Fill in:
   - **Space name**: your app name (e.g., `my-text-toolkit`)
   - **SDK**: Gradio
   - **Hardware**: CPU basic (free)
   - **Visibility**: Public
3. **Create the Space**
4. **Upload your three files** (app.py, requirements.txt, README.md)
5. **Watch the build logs** — look for:
   - `Installing requirements...` — packages being installed
   - `Running on...` — app started successfully
   - Red text = something broke
6. **Test the live URL**

Your app will be at: `https://huggingface.co/spaces/YOUR-USERNAME/YOUR-SPACE-NAME`

### Troubleshooting

| Error | What It Means | Fix |
|-------|--------------|-----|
| `ModuleNotFoundError: No module named 'X'` | Package missing from requirements.txt | Add `X` to requirements.txt |
| `RuntimeError: CUDA out of memory` | Model too large for free tier | Use a smaller model |
| `SyntaxError` on a line | Code error in app.py | Read the line number, fix it |
| `yaml.scanner.ScannerError` | Bad YAML in README.md | Check indentation and `---` delimiters |
| Space shows "Sleeping" | Free-tier Spaces sleep when idle | Normal — wakes up when visited |

### Verification

**My public URL**: 

**Works on my phone?** 

**Classmate's reaction**: 

**Differences between local and deployed version**: 



---

## 6-Question Protocol: The Deployment

Apply the examination to the *deployment experience* — not just the app, but the process.

### 1. What did I ask it to do?


### 2. What did it actually do?


### 3. Where did it succeed?


### 4. Where did it fail or struggle?


### 5. Why might it have failed?


### 6. What would I do differently next time?



---

## Deployment Checklist

Check each item:

- [ ] All imports at top of app.py
- [ ] Inline device detection (no relative imports)
- [ ] Pipelines load at module level
- [ ] requirements.txt lists every package
- [ ] README.md has valid YAML front matter
- [ ] App runs locally before deploying
- [ ] Tested on a different device after deployment
- [ ] Error handling works in production

---

## DCS Question: What Connects This System to Its Users?

Before deployment, your app existed only on your machine. Now a URL connects it to anyone.

**What flows through that URL?**
- *(What does the user send? What do they get back?)*

**What DOESN'T flow through?**
- *(What does the user NOT see? Model limitations? Training data? Your knowledge of edge cases?)*

**Based on your classmate's test of the deployed version**: What did they see that helped them? What information was missing?



---

## Record: CLAUDE.md Week 5 Entry

Add this to your CLAUDE.md file:

```
## Week 5: Deploy

### Interface Upgrade
Upgraded from gr.Interface to gr.Blocks.
Layout changes: [what I added]
Error handling: [what guards I put in]
Why: [what Week 4 user test findings motivated this]

### gr.Blocks Pattern
with gr.Blocks() as demo:
    gr.Markdown("# Title")
    with gr.Row():
        input = gr.Textbox(...)
        output = gr.Textbox(...)
    btn = gr.Button("Go")
    btn.click(fn=func, inputs=input, outputs=output)

### Deployment Notes
Three files: app.py, requirements.txt, README.md
My Space URL: [URL]
What broke: [specific error]
How I fixed it: [specific fix]
Local vs. deployed differences: [observations]

### Deployment Checklist
[Copy from notebook — checked items]

### DCS: What Connects This System to Its Users?
[What flows through the URL. What doesn't.]
```

---

## What's Next

Your app is live. Anyone with the link can use it. But "it works" isn't the same as "it's good."

**Next week**: You design a real user test — not "hey try this" but a systematic test with specific tasks, real testers outside this class, and a framework for turning feedback into decisions. You'll also ask: where does accountability live when this system is wrong?